In [17]:
import starfile
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist,cdist
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def remove_single_entry_micrographs(df,micrographs):
    """
    Remove micrographs that only have one particle associated with them
    """
    for micrograph in micrographs:
        if len(df[df.rlnMicrographName==micrograph])<=1:
            df.drop(df[df.rlnMicrographName==micrograph].index, inplace=True)
    return df

def get_nearest_neighbor_distances(points):
    # Compute the pairwise distances
    distances = cdist(points, points)
    # Set the diagonal to infinity to exclude self-distance
    np.fill_diagonal(distances, np.inf)
    # Find the minimum distance for each point
    nearest_distances = np.min(distances, axis=0)
    return nearest_distances

def plot_nn_dist_mean(df_dist):
    fig,ax=plt.subplots(2,1,figsize=(10,10))
    ax[0].errorbar(df_dist.index-0.1,df_dist.mean_nn_distance,yerr=df_dist.std_nn_distance,label="measured",fmt='o')
    ax[0].errorbar(df_dist.index+0.1,df_dist.random_mean_nn_distance,yerr=df_dist.random_std_nn_distance,label="random",fmt='o')
    ax[0].set_title("Mean nearest neighbor distance")
    ax[0].set_xlabel("Micrograph")
    ax[0].set_ylabel("Distance (pixels)")
    ax[0].legend()
    ax[1].scatter(df_dist.n_particles,df_dist.mean_nn_distance,label="measured")
    ax[1].scatter(df_dist.n_particles,df_dist.random_mean_nn_distance,label="random")
    ax[1].set_title("Mean nearest neighbor distance")
    ax[1].set_xlabel("Number of particles")
    ax[1].set_ylabel("Distance (pixels)")
    ax[1].legend()


    plt.show()

def plot_nn_dist_median(df_dist):
    fig,ax=plt.subplots(2,1,figsize=(10,10))
    ax[0].errorbar(df_dist.index-0.1,df_dist.mean_nn_distance,yerr=df_dist.std_nn_distance,label="measured",fmt='o')
    ax[0].errorbar(df_dist.index+0.1,df_dist.random_mean_nn_distance,yerr=df_dist.random_std_nn_distance,label="random",fmt='o')
    ax[0].set_title("Median nearest neighbor distance")
    ax[0].set_xlabel("Micrograph")
    ax[0].set_ylabel("Distance (pixels)")
    ax[0].legend()
    ax[1].scatter(df_dist.n_particles,df_dist.median_nn_distance,label="measured")
    ax[1].scatter(df_dist.n_particles,df_dist.random_median_nn_distance,label="random")
    ax[1].set_title("Median nearest neighbor distance")
    ax[1].set_xlabel("Number of particles")
    ax[1].set_ylabel("Distance (pixels)")
    ax[1].legend()

def t_test(a,b):
    from scipy.stats import ttest_ind
    t_stat, p_val = ttest_ind(a, b)
    return t_stat, p_val

def ks_test(a,b):
    from scipy.stats import ks_2samp
    ks_stat, p_val = ks_2samp(a, b)
    return ks_stat, p_val

In [ ]:
df=starfile.read('data/particles.star')["particles"]
micrographs=list(df.rlnMicrographName.unique())
df=remove_single_entry_micrographs(df,micrographs)
micrographs=list(df.rlnMicrographName.unique())
df

In [ ]:
ppm=[np.array((df[df.rlnMicrographName==mg].rlnCoordinateX,df[df.rlnMicrographName==mg].rlnCoordinateY)).T for mg in micrographs if len(df[df.rlnMicrographName==mg])>1]



distances=[pdist(arr) for arr in ppm]

nn_dist_measured=[get_nearest_neighbor_distances(arr) for arr in ppm]

df_dist=pd.DataFrame(
    {
        "micrograph":micrographs,
        "n_particles":[arr.shape[0] for arr in ppm],
        "mean_distance":[np.mean(d) for d in distances],
        "sem_distance":[np.std(d)/np.sqrt(len(d)) for d in distances],
        "std_distance":[np.std(d) for d in distances],
        "mean_nn_distance":[np.mean(d) for d in nn_dist_measured],
        "sem_nn_distance":[np.std(d)/np.sqrt(len(d)) for d in nn_dist_measured],
        "std_nn_distance":[np.std(d) for d in nn_dist_measured],
        "median_nn_distance":[np.median(d) for d in nn_dist_measured],
        }
)

df_dist

In [ ]:
all_random_dist=[]

for i, row in df_dist.iterrows():
    arr=np.random.rand(row.n_particles,2)
    arr=np.array((arr[:,0]*(df.rlnCoordinateX.max()-df.rlnCoordinateX.min())+df.rlnCoordinateX.min(),arr[:,1]*(df.rlnCoordinateY.max()-df.rlnCoordinateY.min())+df.rlnCoordinateY.min())).T.astype(int)
    distances=pdist(arr)
    nn_dist=get_nearest_neighbor_distances(arr)
    all_random_dist.append(nn_dist)
    df_dist.loc[i,"random_mean_distance"]=np.mean(distances)
    df_dist.loc[i,"random_sem_distance"]=np.std(distances)/np.sqrt(len(distances))
    df_dist.loc[i,"random_std_distance"]=np.std(distances)
    df_dist.loc[i,"random_mean_nn_distance"]=np.mean(nn_dist)
    df_dist.loc[i,"random_sem_nn_distance"]=np.std(nn_dist)/np.sqrt(len(nn_dist))
    df_dist.loc[i,"random_std_nn_distance"]=np.std(nn_dist)
    df_dist.loc[i,"random_median_nn_distance"]=np.median(nn_dist)
    if i==50:
        arr_rand_plot=arr

fig,ax=plt.subplots(1,2,figsize=(10,10))
ax[0].scatter(ppm[50][:,0],ppm[50][:,1],marker="x", color='k')
ax[0].set_title(f"Measured {ppm[50].shape[0]} particles")
ax[0].set_xlim(0,4096)
ax[0].set_ylim(0,4096)
ax[1].scatter(arr_rand_plot[:,0],arr_rand_plot[:,1],marker="x", color='k')
ax[1].set_title(f"Random {arr_rand_plot.shape[0]} particles")
ax[1].set_xlim(0,4096)
ax[1].set_ylim(0,4096)
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')

plot_nn_dist_mean(df_dist)
plot_nn_dist_median(df_dist)


In [ ]:
meas_nn=np.concat(nn_dist_measured)
rng_nn=np.concat(all_random_dist)

t_stat, p_val = t_test(meas_nn, rng_nn)
print(f"T-test: t_stat={t_stat}, p_val={p_val}, means={np.mean(meas_nn)}, {np.mean(rng_nn)}")
ks_stat, p_val = ks_test(meas_nn, rng_nn)
print(f"KS-test: ks_stat={ks_stat}, p_val={p_val}, medians={np.median(meas_nn)}, {np.median(rng_nn)}")


fig,ax=plt.subplots(1,3,figsize=(15,4))
ax[0].scatter(ppm[50][:,0],ppm[50][:,1],marker="x", color='k')
ax[0].set_title(f"Measured {ppm[50].shape[0]} particles")
ax[0].set_xlim(0,4096)
ax[0].set_ylim(0,4096)
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[1].scatter(arr_rand_plot[:,0],arr_rand_plot[:,1],marker="x", color='k')
ax[1].set_title(f"Random {arr_rand_plot.shape[0]} particles")
ax[1].set_xlim(0,4096)
ax[1].set_ylim(0,4096)
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
sns.violinplot(pd.DataFrame({"Picked Particles":meas_nn,"Random Particles":rng_nn}),ax=ax[2])
sns.despine(ax=ax[2],offset=10)

fig.savefig("results/nn_distances.svg", dpi=300, bbox_inches='tight',transparent=True)